<a href="https://colab.research.google.com/github/ajohn256/Deep-Learning-With-Pytorch/blob/main/Pneumonia_Preditiction_Ready_To_Use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
import os
import torch.nn as nn
from PIL import Image
import torch.nn.functional as F

In [2]:
class XrayClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,12,5)#12,252,252 //input_size(256) - kernal_size(5) + stride(1)
        self.pool = nn.MaxPool2d(2,2)#12,126,126

        self.conv2 = nn.Conv2d(12,24,5)#62-5 +1 >>(12,122,122) -> maxpool2d -> (12,61,61) -> flatten (12*29*29)

        #fully connected layer
        self.fc1 = nn.Linear(24 * 61 * 61, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)



    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # x = x.view(-1, 12 * 29 * 29)
        x = torch.flatten(x,1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [3]:
classifier = XrayClassifier()

In [12]:
result = classifier.load_state_dict(torch.load("./XRAY_Classification_model_UPDATED.pth",map_location=torch.device('cpu')))


<ipython-input-12-5e915505a5ee>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  result = classifier.load_state_dict(torch.load("./XRAY_Classification_model_UPDATED.pth",map

In [13]:
# Check for missing or unexpected keys
if result.missing_keys or result.unexpected_keys:
    print("Warning: Some keys in the state dict were not loaded or were unexpected.")
    print("Missing keys:", result.missing_keys)
    print("Unexpected keys:", result.unexpected_keys)

model = classifier

In [14]:
def predict_image(image):
    transform = transforms.Compose([
                        transforms.Resize((256,256)),
                        transforms.ToTensor(),
                        # Convert the image to RGB format if it's grayscale
                        transforms.Lambda(lambda image: image.repeat(3, 1, 1) if image.shape[0] == 1 else image),
                        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                    ])
    image = Image.open(image).convert('RGB')
    image_transform = transform(image)
    image = image_transform.unsqueeze(0)
    return image

In [15]:
class_names = ['Normal','Pneumonia']

def predict_images(root):
    for path, _ , files in os.walk(root):
        for file in files:
            x = path + "/" + file

            # model.eval()
            with torch.no_grad():
                image = predict_image(x)
                output = model(image)
                _, predicted = torch.max(output,1)
                print(f"Prediction {x} is {class_names[predicted.item()]}")




In [20]:
new_path = "./test"
predict_images(new_path)


Prediction ./test/download (2).jpg is Pneumonia
Prediction ./test/N3.jpg is Normal
Prediction ./test/N2.jpg is Normal
Prediction ./test/OIP.jpg is Pneumonia
Prediction ./test/N1.jpg is Pneumonia
Prediction ./test/download.jpg is Pneumonia
Prediction ./test/th.jpg is Pneumonia
Prediction ./test/download (1).jpg is Normal
